In [ ]:
%matplotlib inline
import os
import numpy as np
filepath='D:/measuring'
import scipy.signal
os.chdir(filepath)


In [ ]:
import analysis.scripts.cavity.oscilloscope_analysis_linewidth as oscillo_ana

In [ ]:

# Parameters for
g_a1 = 0.00
g_A1 = 0.0004
g_x01 = 0.002
g_gamma1 = 0.001
g_dx = 0.0025
g_A2 = 0.0004
#g_x02 = 0.02
g_gamma2 = 0.001


fixed =[]#[0,1,2,5,6]
indir='D:\measuring\data/20160430\OFF_diamond'
filename='OFFD000'
reload(oscillo_ana)
oscillo_ana.plot_and_fit(indir, filename,g_a1, g_A1, g_x01, g_gamma1, g_dx, g_A2, g_gamma2,fixed=fixed)

In [ ]:
reload(oscillo_ana)